<div style="display: table; width: 100%;">
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 70%;">
    <h1>Herramientas para Data Science</h1>
  </div>
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 30%;">
    <img src="https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1/blob/main/Assets/UideLogo.png?raw=true" alt="logo UIDE" style="width:50%;">
  </div>
</div>
<hr />

### 🟦 Componente Práctico 1  
🟡 Grupo: 3      
🟡 Semana: 1      
🟡 Docente:  Ing. Iván García S., PhD. (idgs78@hotmail.com)     

### 🟦 Realizado por:   
Estudiantes

💻 Evelin Rosero Ordoñez   

💻 Marjorie Muso Tandalla

💻 José Espinoza Bone

### 🟦 Objetivo y alcance del trabajo 
- Esta práctica tiene el objetivo de realizar un preprocesamiento de datos y un Análisis 
exploratorio de datos (estadísticas y visualización) de las variables más relevantes 
que considere para el propósito de predicción de cáncer de mama. Al finalizar la 
práctica, los maestrantes podrán manipular adecuadamente un dataset de datos 
estructurados (alfanuméricos). [Breast Cancer Wisconsin (Diagnostic)](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)


### 🟦 [Código fuente original](https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1.git)
Con [git](https://git-scm.com/) instalado. En Windows, Linux o MacOS ejecutar el comando.

```
git clone "https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1.git"
```

# 0️⃣ Preparar entorno

Funciones base para utilizar si son requeridas en el presente notebook. Adicional hay funciones utilitarias para utilizar con pandas.DataFrame y finalmente las funciones para cumplir con los objetivos del presente trabajo práctico.

In [ ]:
# UTILIDADES PARA GESTIÓN DE DEPENDENCIAS E INFORMACIÓN DEL ENTORNO

import sys
import subprocess
import os
from pathlib import Path
from enum import Enum
import zipfile
from typing import Optional
from typing import Iterable
from dataclasses import dataclass
from typing import cast
from typing import Tuple
from types import SimpleNamespace
from typing import Any
from typing import Protocol
from typing import Literal
from typing import Sequence

# Libs a instalar
LIBS = [
    "numpy",
    "pandas",
    "matplotlib",
    "seaborn",
    "scikit-learn",
    "requests",
    "wcwidth",
]

class ConsoleColor(Enum):
    RED = "\033[91m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    BLUE = "\033[94m"
    MAGENTA = "\033[95m"
    CYAN = "\033[96m"
    WHITE = "\033[97m"
    RESET = "\033[0m"


def PrintColor(message: str, color: ConsoleColor) -> str:
    RESET = ConsoleColor.RESET.value
    return f"{color.value}{message}{RESET}"


def ShowMessage(
    message: str, title: str, icon: str, color: ConsoleColor, end: str = "\n"
):
    colored_title = PrintColor(icon + f"  " + title.upper() + ":", color)
    print(f"{colored_title} {message}", end=end)


def ShowInfoMessage(
    message: str, title: str = "Info", icon: str = "ℹ️", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.CYAN, end)


def ShowSuccessMessage(
    message: str, title: str = "Success", icon: str = "✅", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.GREEN, end)


def ShowErrorMessage(
    message: str, title: str = "Error", icon: str = "❌", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.RED, end)


def ShowWarningMessage(
    message: str, title: str = "Warning", icon: str = "⚠️", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.YELLOW, end)


# Funcion para ejecutar comandos
def RunCommand(
    commandList: list[str], printCommand: bool = True, printError: bool = True
) -> subprocess.CompletedProcess[str]:
    print("⏳", " ".join(commandList))

    if printCommand:
        proc = subprocess.Popen(
            commandList,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True,
        )

        out_lines: list[str] = []
        assert proc.stdout is not None
        for line in proc.stdout:
            print(line, end="")
            out_lines.append(line)

        proc.wait()
        err_text = ""
        if proc.stderr is not None:
            err_text = proc.stderr.read() or ""

        if proc.returncode != 0 and printError and err_text:
            ShowErrorMessage(err_text, "", end="")
            # print(err_text, end="")

        return subprocess.CompletedProcess(
            args=commandList,
            returncode=proc.returncode,
            stdout="".join(out_lines),
            stderr=err_text,
        )

    else:
        result = subprocess.run(
            commandList, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        if result.returncode != 0 and printError and result.stderr:
            ShowErrorMessage(result.stderr, "", end="")
            # print(result.stderr, end="")
        return result


# Función para instalar las dependencias
def InstallDeps(libs: Optional[list[str]] = None):
    print("ℹ️ Installing deps.")
    printCommand = False
    printError = True
    RunCommand(
        [sys.executable, "-m", "pip", "install", "--upgrade", "pip"],
        printCommand=printCommand,
        printError=printError,
    )
    if libs is None or libs.count == 0:
        print("No hay elementos a instalar.")
    else:
        RunCommand(
            [sys.executable, "-m", "pip", "install", *libs],
            printCommand=printCommand,
            printError=printError,
        )
        print("Deps installed.")
    print()


# Función para mostrar info el ambiente de ejecución
def ShowEnvironmentInfo():
    print("ℹ️  Environment Info:")
    print("Python Version:", sys.version)
    print("Platform:", sys.platform)
    print("Executable Path:", sys.executable)
    print("Current Working Directory:", os.getcwd())
    print("VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV"))
    print("sys.prefix:", sys.prefix)
    print("sys.base_prefix:", sys.base_prefix)
    print()


InstallDeps(LIBS)
ShowEnvironmentInfo()
import requests


@dataclass(frozen=True)
class BoxStyle:
    TL: str
    TR: str
    BL: str
    BR: str
    H: str
    V: str

class TitleBoxLineStyle(Enum):
    SIMPLE = BoxStyle("┌", "┐", "└", "┘", "─", "│")
    DOUBLE = BoxStyle("╔", "╗", "╚", "╝", "═", "║")
    ROUNDED = BoxStyle("╭", "╮", "╰", "╯", "─", "│")
    HEAVY = BoxStyle("┏", "┓", "┗", "┛", "━", "┃")
    ASCII = BoxStyle("+", "+", "+", "+", "-", "|")
    DOUBLE_BOLD = BoxStyle("╔", "╗", "╚", "╝", "╬", "║")
    BLOCK = BoxStyle("█", "█", "█", "█", "█", "█")
    HEAVY_CROSS = BoxStyle("╒", "╕", "╘", "╛", "╪", "┃")
    METAL = BoxStyle("╞", "╡", "╘", "╛", "═", "║")


# Función para mostrar un título con recuadro
def ShowTitleBox(
    text: str,
    max_len: int = 100,
    boxLineStyle: TitleBoxLineStyle = TitleBoxLineStyle.SIMPLE,
    color: ConsoleColor = ConsoleColor.CYAN,
):
    try:

        def vislen(s: str) -> int:
            from wcwidth import wcswidth as _w

            n = _w(s)
            return n if n >= 0 else len(s)

    except Exception:

        def vislen(s: str) -> int:
            return len(s)

    pad = 1
    tlen = vislen(text)
    inner = max(max_len, tlen)
    left = (inner - tlen) // 2
    right = inner - tlen - left

    top = f"{boxLineStyle.value.TL}{boxLineStyle.value.H * (inner + 2 * pad)}{boxLineStyle.value.TR}"
    mid = f"{boxLineStyle.value.V}{' ' * pad}{' ' * left}{text}{' ' * right}{' ' * pad}{boxLineStyle.value.V}"
    bot = f"{boxLineStyle.value.BL}{boxLineStyle.value.H * (inner + 2 * pad)}{boxLineStyle.value.BR}"
    print(PrintColor("\n".join([top, mid, bot]), color))


# Función para descargar un archivo
def DownloadFile(uri: str, filename: str, overwrite: bool = False, timeout: int = 20, printInfo: bool = True):
    dest = Path(filename).resolve()
    if dest.exists() and dest.is_file() and dest.stat().st_size > 0 and not overwrite:
        if printInfo:
            print(
                f'✅ Ya existe: "{dest}". No se descarga (use overwrite=True para forzar).'
            )
        return
    if dest.parent and not dest.parent.exists():
        dest.parent.mkdir(parents=True, exist_ok=True)
    if printInfo:
        print(f'ℹ️ Descargando "{uri}" → "{dest}"')
    try:
        with requests.get(uri, stream=True, timeout=timeout) as resp:
            resp.raise_for_status()
            tmp = dest.with_suffix(dest.suffix + ".part")
            with open(tmp, "wb") as f:
                for chunk in resp.iter_content(chunk_size=1024 * 64):
                    if chunk:  # filtra keep-alive chunks
                        f.write(chunk)
            tmp.replace(dest)
        if printInfo: 
            print(f'✅ Archivo "{dest}" descargado exitosamente.')
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al descargar: {e}")


# Función para descomprimir un archivo zip
def UnzipFile(filename: str, outputDir: str):
    print(f'ℹ️ Descomprimiendo "{filename}" en "{outputDir}"')
    try:
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(outputDir)
        print(f"Descomprimido en: {os.path.abspath(outputDir)}")
    except Exception as e:
        print(f"Error: {e}")


In [ ]:
# UTILIDADES PARA ANÁLISIS Y MANIPULACIÓN DE DATAFRAMES

# Importar libraries
import pandas as pd
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from matplotlib.figure import Figure
from matplotlib.axes import Axes

from pandas import DataFrame
from pandas import Series
from sklearn.utils import Bunch
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    precision_score, recall_score, f1_score
)

warnings.filterwarnings("ignore")

# Configurar opciones de Pandas
pd.set_option("display.float_format", "{:.2f}".format)
pandas.set_option("display.max_rows", None)
pandas.set_option("display.max_columns", None)


# Función para mostrar la información del DataFrame.
def ShowDfInfo(df: pandas.DataFrame, title):
    display(f"ℹ️ INFO {title} ℹ️")
    df.info()
    display()


# Función para mostrar las n primeras filas del DataFrame.
def ShowDfHead(df: pandas.DataFrame, title: str, headQty=10):
    display(f"ℹ️ {title}: Primeros {headQty} elementos.")
    display(df.head(headQty))
    display()


# Función para mostrar las n últimas filas del DataFrame.
def ShowDfTail(df: pandas.DataFrame, title: str, tailQty=10):
    display(f"ℹ️ {title}: Últimos {tailQty} elementos.")
    display(df.tail(tailQty))
    display()


# Mostrar el tamaño del DataFrame
def ShowDfShape(df: pandas.DataFrame, title: str):
    display(f"ℹ️ {title} - Tamaño de los datos")
    display(f"{df.shape[0]} filas x {df.shape[1]} columnas")
    display()


# Función para mostrar la estadística descriptiva de todas las columnas del DataFrame, por tipo de dato.
def ShowDfStats(df: pandas.DataFrame, title: str = ""):
    display(f"ℹ️ Estadística descriptiva - {title}")
    numeric_cols = df.select_dtypes(include="number")
    if not numeric_cols.empty:
        display("    🔢 Columnas numéricas".upper())
        numeric_desc = (
            numeric_cols.describe().round(2).T
        )  # Transpuesta para añadir columna
        numeric_desc["var"] = numeric_cols.var(numeric_only=True).round(2)
        display(numeric_desc.T)
    non_numeric_cols = df.select_dtypes(
        include=["boolean", "string", "category", "object"]
    )
    if not non_numeric_cols.empty:
        display("    🔡 Columnas no numéricas".upper())
        non_numeric_desc = non_numeric_cols.describe()
        display(non_numeric_desc)
    datetime_cols = df.select_dtypes(include=["datetime", "datetimetz"])
    if not datetime_cols.empty:
        display("    📅 Columnas fechas".upper())
        datetime_desc = datetime_cols.describe()
        display(datetime_desc)


# Función para mostrar una visión general completa del DataFrame
def ShowFullDfOverview(df, title, headQty=5, tailQty=5):
    ShowDfInfo(df, title)
    ShowDfStats(df, title)
    ShowDfShape(df, title)
    ShowDfHead(df, title, headQty=headQty)
    ShowDfTail(df, title, tailQty=tailQty)


# Función para mostrar los valores nulos o NaN de cada columna en un DataFrame
def ShowDfNanValues(df: pandas.DataFrame, title: str):
    display(f"ℹ️ Contador de valores Nulos - {title}")
    nulls_count = df.isnull().sum()
    nulls_df = nulls_count.reset_index()
    nulls_df.columns = ["Columna", "Cantidad_Nulos"]
    display(nulls_df)
    display()


# Tipos de correlación
class CorrelationType(Enum):
    ALL = "all"
    STRONG = "strong"
    WEAK = "weak"


# Muestra las correlaciones completas, débiles y fuertes.
def ShowDfCorrelation(
    df: pandas.DataFrame,
    title: str,
    fig: Optional[Figure] = None,
    ax: Optional[Axes] = None,
    level: CorrelationType = CorrelationType.ALL,
    umbral: float = 0.6,
    showTable: bool = False,
    figsize: tuple = (8, 6),
    annotate: bool = True,
):
    if fig is None or ax is None:
        fig, ax = plt.subplots(figsize=figsize)

    display(f"ℹ️ {(title).upper()} - Matriz de Correlación, Type: {level.name}")

    corr = df.select_dtypes(include=["number"]).corr().copy()

    if level == CorrelationType.STRONG:
        corr = corr.where(np.abs(corr) >= umbral)
    elif level == CorrelationType.WEAK:
        corr = corr.where(np.abs(corr) < umbral)
        np.fill_diagonal(corr.values, 1)
    elif level != CorrelationType.ALL:
        raise ValueError(f"Invalid level: {level}")

    cax = ax.matshow(corr, vmin=-1, vmax=1)

    cols = corr.columns
    ax.set_xticks(range(len(cols)))
    ax.set_yticks(range(len(cols)))
    ax.set_xticklabels(cols, rotation=90, ha="left")
    ax.set_yticklabels(cols)

    fig.colorbar(cax)

    if annotate:
        for (i, j), value in np.ndenumerate(corr.values):
            if not np.isnan(value):
                ax.text(j, i, f"{value:+.2f}", ha="center", va="center")

    if level == CorrelationType.ALL:
        titulo = "Matriz de correlación completa"
    else:
        titulo = f"Matriz de correlación ({level.name}, umbral={umbral})"

    total_elementos = corr.size
    total_nodiagonal = corr.size - corr.shape[0]
    total_nan = corr.isna().sum().sum()
    total_validos = total_elementos - total_nan - corr.shape[0]

    titulo = (
        f"{titulo}, Total Matriz: {total_nodiagonal}, "
        f"Total válidos: {total_validos} ({((total_validos * 100) / total_nodiagonal):.2f}%)"
    )

    ax.set_title(titulo, pad=20)
    ax.grid(False)
    plt.tight_layout()
    plt.show()

    if showTable:
        display(corr)
    fig.show()
    return fig, corr


def NormalizeColumnNames(df: pandas.DataFrame) -> pandas.DataFrame:
    df.columns = [
        col.strip().title().replace(" ", "").replace("_", "") for col in df.columns
    ]
    return df


def DropColumns(
    df: pandas.DataFrame, toDrop: list[str], inplace: bool = False
) -> pandas.DataFrame:
    if not toDrop:
        return df
    if inplace:
        df.drop(columns=df.columns.intersection(toDrop), inplace=True)
        return df
    else:
        return df.drop(columns=df.columns.intersection(toDrop))


# Para almacenar los datos del dataset
@dataclass
class Dataset:
    X: pandas.DataFrame
    y: pandas.DataFrame


# Para almacenar los datos de split del dataset.
@dataclass
class DatasetSplit:
    Train: Dataset
    Test: Dataset


# Muestra el head de cada componente del split.
def ShowDatasetSplitHead(split: DatasetSplit, title: str, headQty: int = 5):
    ShowDfHead(split.Train.X, f"{title} - X Train", headQty)
    ShowDfHead(split.Train.y, f"{title} - y Train", headQty)
    ShowDfHead(split.Test.X, f"{title} - X Test", headQty)
    ShowDfHead(split.Test.y, f"{title} - y Test", headQty)


# Muestra la información del Dataset
def ShowDatasetInfo(data: Dataset, title):
    tAux = title
    title = f"{tAux} - Caracteristicas - X"
    ShowDfInfo(data.X, title)
    ShowDfShape(data.X, title)
    ShowDfStats(data.X, title)
    ShowDfNanValues(data.X, title)
    ShowDfHead(data.X, title)
    ShowDfTail(data.X, title)
    title = f"{tAux} - Características - y"
    ShowDfInfo(data.y, title)
    ShowDfShape(data.y, title)
    ShowDfStats(data.y, title)
    ShowDfNanValues(data.y, title)
    ShowDfHead(data.y, title)
    ShowDfTail(data.y, title)


# Muestra la información del Dataset Split
def ShowDatasetSplitInfo(split: DatasetSplit, title: str, headQty: int = 5):
    tAux = title
    title = f"{tAux} - TRAIN"
    ShowDatasetInfo(split.Train, title)
    title = f"{tAux} - TEST"
    ShowDatasetInfo(split.Test, title)


# Realiza el split del Dataset, en Train y test utilizando el ratio.
def SplitDataset(
    data: Dataset, trainRatio: float = 0.8, randomState: int = 42
) -> DatasetSplit:
    y_strat = data.y.iloc[:, 0]
    XTrain, XTest, yTrain, yTest = train_test_split(
        data.X,
        data.y,
        train_size=trainRatio,
        random_state=randomState,
        stratify=y_strat,
    )
    return DatasetSplit(
        Train=Dataset(X=XTrain.reset_index(drop=True), y=yTrain.reset_index(drop=True)),
        Test=Dataset(X=XTest.reset_index(drop=True), y=yTest.reset_index(drop=True)),
    )


# Contrato para los escaladores
class ScalerProtocol(Protocol):
    def fit(self, X, y: Any = None) -> Any: ...
    def transform(self, X) -> Any: ...
    def fit_transform(self, X, y: Any = None) -> Any: ...


# Para almacenar los datos del dataset aplicado el escalador.
@dataclass
class ScaledDatasetSplit(DatasetSplit):
    Scaler: ScalerProtocol

# Enum para los tipos de escaladores soportados
class ScalerType(Enum):
    STANDARD = "Standard"
    MIN_MAX = "minmax"
    ROBUST = "robust"
    MAX_ABS = "maxabs"
    NORMALIZER = "normalizer"
    QUANTILE = "quantile"
    POWER = "power"
    FUNCTION = "function"


# Crea una instancia de scaler según el Enum ScalerType.
def CreateScaler(scalerType: ScalerType, **kwargs) -> ScalerProtocol:
    if scalerType == ScalerType.STANDARD:
        return StandardScaler(**kwargs)
    if scalerType == ScalerType.MIN_MAX:
        return MinMaxScaler(**kwargs)
    if scalerType == ScalerType.ROBUST:
        return RobustScaler(**kwargs)
    if scalerType == ScalerType.MAX_ABS:
        return MaxAbsScaler(**kwargs)
    if scalerType == ScalerType.NORMALIZER:
        return Normalizer(**kwargs)
    if scalerType == ScalerType.QUANTILE:
        return QuantileTransformer(**kwargs)
    if scalerType == ScalerType.POWER:
        return PowerTransformer(**kwargs)
    if scalerType == ScalerType.FUNCTION:
        return FunctionTransformer(**kwargs)
    raise ValueError(f"ScalerType no soportado: {scalerType}")

def DetectScaler(scaler: ScalerProtocol) -> ScalerType:
    if isinstance(scaler, StandardScaler):
        return ScalerType.STANDARD
    if isinstance(scaler, MinMaxScaler):
        return ScalerType.MIN_MAX
    if isinstance(scaler, RobustScaler):
        return ScalerType.ROBUST
    if isinstance(scaler, MaxAbsScaler):
        return ScalerType.MAX_ABS
    if isinstance(scaler, Normalizer):
        return ScalerType.NORMALIZER
    if isinstance(scaler, QuantileTransformer):
        return ScalerType.QUANTILE
    if isinstance(scaler, PowerTransformer):
        return ScalerType.POWER
    if isinstance(scaler, FunctionTransformer):
        return ScalerType.FUNCTION
    raise ValueError(f"No se reconoce el tipo de scaler: {type(scaler)}")

# Escala el split usando el escalador proporcionado y retorna el split escalado.
def ScaleDatasetSplit(
    split: DatasetSplit, scaler: ScalerProtocol = StandardScaler()
) -> ScaledDatasetSplit:
    XTrainScaledValues = scaler.fit_transform(split.Train.X)
    XTestScaledValues = scaler.transform(split.Test.X)

    XTrainScaled = pandas.DataFrame(
        XTrainScaledValues, columns=split.Train.X.columns, index=split.Train.X.index
    )

    XTestScaled = pandas.DataFrame(
        XTestScaledValues, columns=split.Test.X.columns, index=split.Test.X.index
    )

    TrainScaledDataset = Dataset(X=XTrainScaled, y=split.Train.y.copy())
    TestScaledDataset = Dataset(X=XTestScaled, y=split.Test.y.copy())

    return ScaledDatasetSplit(
        Train=TrainScaledDataset, Test=TestScaledDataset, Scaler=scaler
    )

# Para almacenar los datos del dataset aplicado PCA.
@dataclass
class PcaDatasetSplit(DatasetSplit):
    Pca: PCA
    Scaler: ScalerProtocol | None = None 

# Aplica PCA al split escalado y retorna el split con PCA aplicado.
def ApplyPCA(
    split: ScaledDatasetSplit,
    explainedVarianceRatioSum: float = 0.95,
    randomState: int = 42
) -> PcaDatasetSplit:

    def GetPCNames(n: int) -> list[str]:
        return [f"PC{i}" for i in range(1, n + 1)]

    pca = PCA(n_components=explainedVarianceRatioSum, random_state=randomState)

    XTrainPCA = pca.fit_transform(split.Train.X)
    XTestPCA = pca.transform(split.Test.X)

    XTrainPcaDf = pandas.DataFrame(
        XTrainPCA, index=split.Train.X.index, columns=GetPCNames(XTrainPCA.shape[1])
    )

    XTestPcaDf = pandas.DataFrame(
        XTestPCA, index=split.Test.X.index, columns=GetPCNames(XTestPCA.shape[1])
    )

    return PcaDatasetSplit(
        Train=Dataset(X=XTrainPcaDf, y=split.Train.y.copy()),
        Test=Dataset(X=XTestPcaDf, y=split.Test.y.copy()),
        Pca=pca,
        Scaler=split.Scaler
    )


# Para almacenar los resultados de la regresión logística
@dataclass
class LogisticRegressionResult:
    Model: LogisticRegression
    Predictions: pandas.DataFrame
    Accuracy: float
    Precision: float
    Recall: float
    F1: float
    ConfusionMatrix: np.ndarray
    Iters: list[int] | np.ndarray
    Report: str
    Target: str

# Aplica regresión logística al split escalado y retorna el resultado.
def ApplyLogisticRegression(
    split: ScaledDatasetSplit,
    targetColumn: str,
    randomState: int = 42,
    maxIter: int = 200,
    C: float = 1.0,
    penalty: Literal['l1', 'l2', 'elasticnet'] | None = "l2",
    solver: Literal['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'] = "lbfgs"
) -> LogisticRegressionResult:
    
    XTrain = split.Train.X
    yTrain = split.Train.y[targetColumn]
    XTest = split.Test.X
    yTest = split.Test.y[targetColumn]

    model = LogisticRegression(
        max_iter=maxIter,
        random_state=randomState,
        C=C,
        penalty=penalty,
        solver=solver
    )

    model.fit(XTrain, yTrain)

    yPredTrain = model.predict(XTrain)
    yPredTest = model.predict(XTest)

    yProbaTrain = model.predict_proba(XTrain)
    yProbaTest = model.predict_proba(XTest)

    dfProbaTest = pandas.DataFrame(
        yProbaTest,
        index=XTest.index,
        columns=[f"Class-{cls}-Prob" for cls in model.classes_]
    )

    dfPredTest = pandas.DataFrame(
        {
            "yReal": yTest.values,
            "yPred": yPredTest
        },
        index=XTest.index
    )

    dfPredTest = pandas.concat([dfPredTest, dfProbaTest], axis=1)

    acc = accuracy_score(yTest, yPredTest)
    prec = precision_score(yTest, yPredTest, average="weighted", zero_division=0)
    rec = recall_score(yTest, yPredTest, average="weighted", zero_division=0)
    f1 = f1_score(yTest, yPredTest, average="weighted", zero_division=0)
    cm = confusion_matrix(yTest, yPredTest)
    report = classification_report(yTest, yPredTest)

    return LogisticRegressionResult(
        Model=model,
        Predictions=dfPredTest,
        Accuracy=float(acc),
        ConfusionMatrix=cm,
        Precision=float(prec),
        Recall=float(rec),
        F1=float(f1),
        Iters=model.n_iter_,
        Report=str(report),
        Target=targetColumn
    )

# Para almacenar los resultados de KNN
@dataclass
class KnnResult:
    Model: KNeighborsClassifier
    Predictions: pandas.DataFrame
    Accuracy: float
    Precision: float
    Recall: float
    F1: float
    ConfusionMatrix: np.ndarray
    Report: str
    Target: str
    K: int

def GetBestKForKNN(
    split: ScaledDatasetSplit,
    targetColumn: str,
    kMax: int = 30
) -> int:
    
    XTrain = split.Train.X
    yTrain = split.Train.y[targetColumn]
    XTest = split.Test.X
    yTest = split.Test.y[targetColumn]

    bestK = 1
    bestScore = -1.0

    for k in range(1, kMax + 1):
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(XTrain, yTrain)

        yPred = model.predict(XTest)
        acc = accuracy_score(yTest, yPred)

        if acc > bestScore:
            bestScore = acc
            bestK = k
    return bestK

def ApplyKNN(
    split: ScaledDatasetSplit,
    targetColumn: str,
    weights: Literal["uniform", "distance"] = "uniform",
    metric: Literal[
        "euclidean", "manhattan", "chebyshev", "minkowski"
    ] = "minkowski"
) -> KnnResult:
    K = GetBestKForKNN(split, targetColumn)
    XTrain = split.Train.X
    yTrain = split.Train.y[targetColumn]
    XTest = split.Test.X
    yTest = split.Test.y[targetColumn]

    model = KNeighborsClassifier(
        n_neighbors=K,
        weights=weights,
        metric=metric
    )

    model.fit(XTrain, yTrain)

    yPred = model.predict(XTest)

    if hasattr(model, "predict_proba"):
        yProba = model.predict_proba(XTest)
        dfProba = pandas.DataFrame(
            yProba,
            index=XTest.index,
            columns=[f"Class-{cls}-Prob" for cls in model.classes_],
        )
    else:
        dfProba = pandas.DataFrame(index=XTest.index)

    dfPred = pandas.DataFrame(
        {
            "yReal": yTest.values,
            "yPred": yPred,
        },
        index=XTest.index
    )

    dfPred = pandas.concat([dfPred, dfProba], axis=1)

    acc = accuracy_score(yTest, yPred)
    prec = precision_score(yTest, yPred, average="weighted", zero_division=0)
    rec = recall_score(yTest, yPred, average="weighted", zero_division=0)
    f1 = f1_score(yTest, yPred, average="weighted", zero_division=0)
    cm = confusion_matrix(yTest, yPred)
    report = classification_report(yTest, yPred)

    return KnnResult(
        Model=model,
        Predictions=dfPred,
        Accuracy=float(acc),
        Precision=float(prec),
        Recall=float(rec),
        F1=float(f1),
        ConfusionMatrix=cm,
        Report=str(report),
        Target=targetColumn,
        K=K
    )


def PlotConfusionMatrix(
    cm,
    classNames: Sequence[str] | None = None,
    title: str = "Confusion Matrix"
):
    # Si classNames es None → usar "auto"
    xticks = classNames if classNames is not None else "auto"
    yticks = classNames if classNames is not None else "auto"

    plt.figure(figsize=(6, 5))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=xticks,
        yticklabels=yticks
    )
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(title)
    plt.tight_layout()
    plt.show()

# Utilidades para detección de tipos de split
@dataclass(frozen=True)
class SplitTypeInfo:
    IsPCA: bool
    IsScaled: bool
    IsRaw: bool

# Detecta el tipo de split (PCA, Escalado, Crudo)
def DetectSplitType(split) -> SplitTypeInfo:
    isPca = isinstance(split, PcaDatasetSplit)
    isScaled = isinstance(split, ScaledDatasetSplit)
    isRaw = not isPca and not isScaled

    return SplitTypeInfo(
        IsPCA=isPca,
        IsScaled=isScaled,
        IsRaw=isRaw
    )

def GenerateMetricsTable(splitList: list[DatasetSplit], targetColumn:str) -> pandas.DataFrame:
    results = []

    for idx, split in enumerate(splitList, start=1):

        # Detectar tipo de split
        splitInfo = DetectSplitType(split)

        # Nombre del scaler o None
        scalerName = (
            DetectScaler(split.Scaler).value
            if (splitInfo.IsScaled and hasattr(split, "Scaler") and split.Scaler is not None)
            else "None"
        )

        # Tipo del split
        if splitInfo.IsPCA:
            splitType = "PCA"
        elif splitInfo.IsScaled:
            splitType = "SCALED"
        else:
            splitType = "RAW"

        # Ejecutar modelos
        resultLR = ApplyLogisticRegression(split, targetColumn=targetColumn)
        resultKNN = ApplyKNN(split, targetColumn=targetColumn)

        # Guardar info en lista
        results.append({
            "Index": idx,
            "Type": splitType,
            "Scaler": scalerName,
            # Métricas Logistic Regression
            "LR_Accuracy": resultLR.Accuracy,
            "LR_Precision": resultLR.Precision,
            "LR_Recall": resultLR.Recall,
            "LR_F1": resultLR.F1,
            # Métricas KNN
            "KNN_BestK": resultKNN.K,
            "KNN_Accuracy": resultKNN.Accuracy,
            "KNN_Precision": resultKNN.Precision,
            "KNN_Recall": resultKNN.Recall,
            "KNN_F1": resultKNN.F1,
        })

    # Convertir a DataFrame final
    df = pandas.DataFrame(results)

    return df


def EvaluateSplits(splitList: list[DatasetSplit], targetColumn: str):

    i = 0
    for split in splitList:
        splitInfo = DetectSplitType(split)
        i += 1

        title = (
            f"{i}.- Data - Scaled = {splitInfo.IsScaled} - PCA = {splitInfo.IsPCA} "
            f"- Features({split.Train.X.columns.size}): "
            + ", ".join(split.Train.X.columns.tolist())
            + ", Scaler: "
            + (
                DetectScaler(split.Scaler).value
                if (splitInfo.IsScaled and hasattr(split, 'Scaler') and split.Scaler is not None)
                else "None"
            )
        )

        ShowTitleBox(title, color=ConsoleColor.CYAN, boxLineStyle=TitleBoxLineStyle.SIMPLE)
        ShowDfInfo(split.Train.X, f"{title}")
        ShowDfHead(split.Train.X, f"{title}", headQty=5)

        result1 = ApplyLogisticRegression(split, targetColumn=targetColumn)
        result2 = ApplyKNN(split, targetColumn=targetColumn)

        ShowTitleBox("RESULTADOS REGRESIÓN LOGÍSTICA",
                     color=ConsoleColor.YELLOW,
                     boxLineStyle=TitleBoxLineStyle.HEAVY)

        print(f"Accuracy: {result1.Accuracy:.4f}")
        print(f"Precision: {result1.Precision:.4f}")
        print(f"Recall: {result1.Recall:.4f}")
        print(f"F1 Score: {result1.F1:.4f}")

        print("Confusion Matrix:")
        PlotConfusionMatrix(
            result1.ConfusionMatrix,
            classNames=result1.Model.classes_,
            title="Logistic Regression - Confusion Matrix"
        )

        print("Classification Report:")
        print(result1.Report)

        ShowTitleBox("RESULTADOS KNN",
                     color=ConsoleColor.YELLOW,
                     boxLineStyle=TitleBoxLineStyle.HEAVY)

        print(f"Best K: {result2.K}")
        print(f"Accuracy: {result2.Accuracy:.4f}")
        print(f"Precision: {result2.Precision:.4f}")
        print(f"Recall: {result2.Recall:.4f}")
        print(f"F1 Score: {result2.F1:.4f}")

        print("Confusion Matrix:")
        PlotConfusionMatrix(
            result2.ConfusionMatrix,
            classNames= result2.Model.classes_,
            title="KNN - Confusion Matrix"
        )

        print("Classification Report:")
        print(result2.Report)

# BREAST CANCER WISCONSIN (DIAGNOSTIC) DATASET 1️⃣ Business Understanding - Entender el negocio

# Breast Cancer Wisconsin (Diagnostic) — Descripción de Columnas

El dataset contiene medidas computadas a partir de imágenes digitalizadas de tumores mamarios obtenidos mediante biopsia por aspiración con aguja fina (FNA).  
Incluye 569 observaciones y 32 columnas.

---

## 🧭 Columnas de Identificación y Diagnóstico

### **1. id**
Identificador único del registro. No se utiliza para entrenamiento.

### **2. diagnosis**
Diagnóstico final del tumor:
- **M** = Maligno  
- **B** = Benigno  

Es la **variable objetivo** del dataset.

---

## 🔬 Variables morfológicas del tumor

Cada variable aparece en tres versiones:

- **mean**: valor promedio  
- **se**: error estándar  
- **worst**: peor valor (percentil 90 o máximo observado)

---

## ⭐ Radius
Radio promedio del tumor, calculado como la distancia del centro al borde del núcleo.
- radius_mean  
- radius_se  
- radius_worst  

## ⭐ Texture
Desviación estándar de valores de intensidad de pixeles. Describe heterogeneidad.
- texture_mean  
- texture_se  
- texture_worst  

## ⭐ Perimeter
Perímetro del núcleo celular.
- perimeter_mean  
- perimeter_se  
- perimeter_worst  

## ⭐ Area
Área del núcleo celular, medida en píxeles.
- area_mean  
- area_se  
- area_worst  

## ⭐ Smoothness
Variación local del radio. Describe qué tan suave o irregular es la forma.
- smoothness_mean  
- smoothness_se  
- smoothness_worst  

## ⭐ Compactness
Calculado como:  
`(perimeter² / area) − 1.0`  
Indica cuán compacta es la forma.
- compactness_mean  
- compactness_se  
- compactness_worst  

## ⭐ Concavity
Grado de concavidad del borde del núcleo.
- concavity_mean  
- concavity_se  
- concavity_worst  

## ⭐ Concave Points
Número de puntos con concavidad en el borde.
- concave points_mean  
- concave points_se  
- concave points_worst  

## ⭐ Symmetry
Mide cuán simétrica es la forma de la célula.
- symmetry_mean  
- symmetry_se  
- symmetry_worst  

## ⭐ Fractal Dimension
Aproximación a la dimensión fractal del borde celular.
- fractal_dimension_mean  
- fractal_dimension_se  
- fractal_dimension_worst  

---


# BREAST CANCER WISCONSIN (DIAGNOSTIC) DATASET 2️⃣ Data preparation - Preparación de los datos
- Utilizando la función `load_breast_cancer` de ScikitLearn podemos obtener el dataset de Breast Cancer Wisconsin.
- Obtenemos el Dataset en "X" las columnas sin la columna objetivo y en "y" la columna a predecir; con el objetivo de entrenar modelos y realizar las predicciones.
- Luego hacemos split del Dataset con un 80% para Train y 20% para Test.
- Como son datos en diferentes escalas procedemos a escalar los valores utilizando un StandardScaler.
- Finalmente mostramos las tablas de correlación completa, fuerte y débil con un umbral de 0.4. Teniendo un 43% de correlaciones fuertes y tiene sentido ya que mean, se y worst tienen relación directa.
- 
- Quedan listos los datos listos para utilizar en un entrenamiento con un algoritmo supervisado y posteriormente hacer los tests.

In [ ]:

# Carga y preprocesa el dataset de Breast Cancer Wisconsin (Diagnostic).
def LoadBreastCancerDataset(toInclude: list[str] | None = None) -> Dataset:
    bc = cast(Bunch, load_breast_cancer(as_frame=True))
    df: DataFrame = bc.frame.copy()

    TARGET_NAME = "target"

    # Separar X e y
    X = df.drop(columns=[TARGET_NAME]).copy()

    # Normalizar nombres de columnas
    NormalizeColumnNames(X)

    # Filtrar únicamente columnas especificadas en toInclude
    if toInclude is not None and len(toInclude) > 0:
        # Tomar solo columnas existentes
        colsToKeep = [c for c in toInclude if c in X.columns]
        X = X[colsToKeep]

    # Target invertido
    y = 1 - df[[TARGET_NAME]].copy()
    y.columns = ["Diagnosis"]

    return Dataset(X, y)


ShowTitleBox("BREAST CANCER WISCONSIN (DIAGNOSTIC) DATASET", color=ConsoleColor.MAGENTA, boxLineStyle= TitleBoxLineStyle.BLOCK)
ShowTitleBox("CARGANDO EL DATASET", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
breastCancerDataset = LoadBreastCancerDataset()

ShowTitleBox("HACIENDO SPLIT AL DATASET", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
breastCancerSplit: DatasetSplit = SplitDataset(breastCancerDataset, trainRatio=0.8)
title = "Split del Dataset"
ShowDatasetSplitInfo(breastCancerSplit, title)

ShowTitleBox("HACIENDO ESCALADO AL SPLIT", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
breastCancerScaledSplit = ScaleDatasetSplit(breastCancerSplit)
title = "Split escalado"
ShowDatasetSplitHead(breastCancerScaledSplit, title)

ShowTitleBox("MATRIZ DE CORRELACIÓN", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
CORR_UMBRAL = 0.4
title = "Breast Cancer Wisconsin (Diagnostic) Dataset - Original sin split"
_ = ShowDfCorrelation(breastCancerDataset.X, title, level=CorrelationType.ALL, showTable=True, figsize=(20,20), annotate=True, umbral=CORR_UMBRAL)
_ = ShowDfCorrelation(breastCancerDataset.X, title, level=CorrelationType.STRONG, showTable=False, figsize=(20,20), annotate=True, umbral=CORR_UMBRAL)
_ = ShowDfCorrelation(breastCancerDataset.X, title, level=CorrelationType.WEAK, showTable=False, figsize=(20,20), annotate=True, umbral=CORR_UMBRAL)



# BREAST CANCER WISCONSIN (DIAGNOSTIC) DATASET 3️⃣ y 4️⃣ Modeling - Evaluating - Aplicación del modelo y evaluación

In [ ]:
cols1 = [
    "MeanRadius",
    "MeanTexture",
    "MeanPerimeter",
    "MeanArea",
    "MeanSmoothness",
    "MeanCompactness",
    "MeanConcavity",
    "MeanConcavePoints",
    "MeanSymmetry",
    "MeanFractalDimension"
]

cols2 = [
    "RadiusError",
    "TextureError",
    "PerimeterError",
    "AreaError",
    "SmoothnessError",
    "CompactnessError",
    "ConcavityError",
    "ConcavePointsError",
    "SymmetryError",
    "FractalDimensionError"
]

cols3 = [
    "WorstRadius",
    "WorstTexture",
    "WorstPerimeter",
    "WorstArea",
    "WorstSmoothness",
    "WorstCompactness",
    "WorstConcavity",
    "WorstConcavePoints",
    "WorstSymmetry",
    "WorstFractalDimension"
]

split1 = SplitDataset( LoadBreastCancerDataset())
split2 = ScaleDatasetSplit( SplitDataset( LoadBreastCancerDataset()))
split3 = ScaleDatasetSplit( SplitDataset( LoadBreastCancerDataset()), CreateScaler(ScalerType.MIN_MAX))
split4 = ApplyPCA( ScaleDatasetSplit( SplitDataset( LoadBreastCancerDataset())))
split5 = ApplyPCA( ScaleDatasetSplit( SplitDataset( LoadBreastCancerDataset()), CreateScaler(ScalerType.MIN_MAX)))
split6 = SplitDataset( LoadBreastCancerDataset(toInclude=cols1))
split7 = SplitDataset( LoadBreastCancerDataset(toInclude=cols2))
split8 = SplitDataset( LoadBreastCancerDataset(toInclude=cols3))

allSplits = [split1, split2, split3, split4, split5, split6, split7, split8]

EvaluateSplits(allSplits, targetColumn="Diagnosis")
GenerateMetricsTable(allSplits, targetColumn="Diagnosis")

# 🏁 Resultados 🏁

Se tiene 8 configuraciones distintas del dataset para probar:

Dataset original (sin escalar)

Dataset escalado (StandardScaler)

Dataset escalado (MinMaxScaler)

PCA aplicado sobre StandardScaler

PCA aplicado sobre MinMaxScaler

Solo primeras 10 columnas (Mean)

Solo columnas de error (Error)

Solo columnas “Worst”

Y para cada uno, se calcula:

- Regresión Logística
- KNN

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Index</th>
      <th>Type</th>
      <th>Scaler</th>
      <th>LR_Accuracy</th>
      <th>LR_Precision</th>
      <th>LR_Recall</th>
      <th>LR_F1</th>
      <th>KNN_BestK</th>
      <th>KNN_Accuracy</th>
      <th>KNN_Precision</th>
      <th>KNN_Recall</th>
      <th>KNN_F1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>RAW</td>
      <td>None</td>
      <td>0.94</td>
      <td>0.94</td>
      <td>0.94</td>
      <td>0.94</td>
      <td>11</td>
      <td>0.95</td>
      <td>0.95</td>
      <td>0.95</td>
      <td>0.95</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>
      <td>SCALED</td>
      <td>Standard</td>
      <td>0.96</td>
      <td>0.97</td>
      <td>0.96</td>
      <td>0.96</td>
      <td>5</td>
      <td>0.96</td>
      <td>0.96</td>
      <td>0.96</td>
      <td>0.96</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>SCALED</td>
      <td>minmax</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>10</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>0.97</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>PCA</td>
      <td>None</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>5</td>
      <td>0.96</td>
      <td>0.96</td>
      <td>0.96</td>
      <td>0.96</td>
    </tr>
    <tr>
      <th>4</th>
      <td>5</td>
      <td>PCA</td>
      <td>None</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>5</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>0.97</td>
      <td>0.97</td>
    </tr>
    <tr>
      <th>5</th>
      <td>6</td>
      <td>RAW</td>
      <td>None</td>
      <td>0.89</td>
      <td>0.89</td>
      <td>0.89</td>
      <td>0.89</td>
      <td>10</td>
      <td>0.88</td>
      <td>0.88</td>
      <td>0.88</td>
      <td>0.87</td>
    </tr>
    <tr>
      <th>6</th>
      <td>7</td>
      <td>RAW</td>
      <td>None</td>
      <td>0.87</td>
      <td>0.88</td>
      <td>0.87</td>
      <td>0.86</td>
      <td>5</td>
      <td>0.89</td>
      <td>0.90</td>
      <td>0.89</td>
      <td>0.89</td>
    </tr>
    <tr>
      <th>7</th>
      <td>8</td>
      <td>RAW</td>
      <td>None</td>
      <td>0.94</td>
      <td>0.94</td>
      <td>0.94</td>
      <td>0.94</td>
      <td>5</td>
      <td>0.94</td>
      <td>0.94</td>
      <td>0.94</td>
      <td>0.94</td>
    </tr>
  </tbody>
</table>
</div>

Escalando y aplicando PCA obtenemos buena precisión(Accuracy) tanto en LogisticRegression como en KNN (0.94 - 0.97).

Todas las combinaciones con escalado con el score F1 se comportan bien teniendo valores entre 0.94 y 0.97.

Utilizar ciertas columnas con son los splits 6,7 y 8 sin escalar dan el peor rendimiento. Al quitar columnas se quita parte del significado de los datos y puede hacer que el modelo falle.

Utilizando MinMaxScaler en todos los scores tiene buenas puntuaciones. Dando como ganador el utilizar MinMaxScaler para este dataset en específico.

Al usar PCA aunque los datos pierdan sentido, pero se mantiene la calidad de los datos a pesar de reducir la dimensionalidad. Se logra detectar tumores malignos. Aplicar PCA nos ayuda a evitar la multicolinealidad en los datos. Por lo que es buena opción y recomendado en este dataset. 

PCA con MinMaxScaler es el ganador definitivo para este dataset.

 